In [ ]:
#Colab setting
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/DSAIL')

import os
os.chdir('/content/drive/My Drive/DSAIL')

Mounted at /content/drive


In [ ]:
# rating 5
import numpy as np
import pandas as pd

test_path = "data/NCF/ml-1m.test.rating"
test_neg_path = "data/NCF/ml-1m.test.negative"
# positive 1 : negative 99 (??)
train_path = "data/NCF/ml-1m.train.rating"


In [ ]:
# user item rating timestamp(?)라서 0, 1번째 col만 사용
train_data = pd.read_csv(train_path, sep='\t', names=['user', 'item'], usecols=[0, 1])
train_data

,user,item
0,0,32
1,0,34
2,0,4
3,0,35
4,0,30
...,...,...
994164,6039,1092
994165,6039,41
994166,6039,128
994167,6039,323


In [ ]:
train_data.dtypes

,0
user,int64
item,int64


In [ ]:
user_num = 6040
item_num = train_data['item'].max() + 1 # 3706
# print(item_num)
train_data_list = train_data.values.tolist()
train_mat = np.zeros((user_num, item_num))
for user, item in train_data_list:
    train_mat[user, item] = 1

3706


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random

class NCF():
  def __init__(self, model_type ='MLP', train_R=train, num_layers=4, test_R=test, d=16, dropout = 0.3, learning_rate=1e-3, epochs= 100):
    '''
    model_type: 'MLP', 'GMF', 'pretrained'
    GMF(Generalized version of MF)
    - yhat = aout(h^(pu * qi))
    - aout: sigmoid

    MLP
    - yhat = sigmoid(hTpiL(ZL-1))
    FusedModel

    '''
    self.num_user, self.num_item = train_R.shape
    self.num_layers = num_layers
    self.drop_out = dropout
    self.dimension = d
    self.model_type = model_type
    self.epochs = epochs

    # implicit하게 변환
    self.train_R = (train_R > 0).astype(float)
    self.test_R = (test_R > 0).astype(float)

    # GMF
    '''
    저자의 코드에서 보면 user와 item latent dimension 동일하게 사용
    하지만 Fused model 한계점으로는 latent dimension 통일을 지적,,, (????)
    '''
    self.user_embedding_gmf = nn.Embedding(self.num_user, d)
    self.item_embedding_gmf = nn.Embedding(self.num_item, d)

    # MLP
    self.user_embedding_mlp = nn.Embedding(self.num_user, d * (2**(num_layers - 1)))
    self.item_embedding_mlp = nn.Embedding(self.num_item, d * (2**(num_layers - 1)))

    mlp_layers = []
    for layer in range(self.num_layers):
      input_size = d * (2** (num_layers - layer))
      mlp_layers.append(nn.Dropout(self.drop_out))
      mlp_layers.append(nn.Linear(input_size, input_size // 2))
      mlp_layers.append(nn.ReLU())

    self.mlp_layers = nn.Sequential(*mlp_layers)

    # GMF든 MLP든 마지막에 Sigmoid 태워주면 됨!!
    if self.model_type == 'GMF' or self.model_type == 'MLP':
      last_size = d
    else:
      # Fused model은 concat해서
      last_size = d * 2
    self.last_layer = nn.Linear(last_size, 1)
    self._init_weight_()

    # 파이썬 오랜만에 짜보는데 넘 예쁘다,,
  def _init_weight_(self):
    if not self.model == 'pretrained':
      nn.init.normal_(self.user_embedding_gmf, std=0.01)
      nn.init.normal_(self.item_embedding_gmf, std=0.01)
      nn.init.normal_(self.user_embedding_mlp, std=0.01)
      nn.init.normal_(self.item_embedding_mlp, std=0.01)

      for layer in self.mlp_layers:
        if isinstance(layer, nn.Linear):
          nn.init.xavier_uniform_(layer.weight)

      # 결국 마지막엔 Sigmoid
      nn.init.kaiming_uniform_(self.last_layer.weight, a=1, nonlinearity = 'sigmoid')

      for m in self.modules():
        if isinstance(m, nn.Linear) and m.bias is not None:
          m.bias.data.zero_()
    else:
      # pretrained 애들 불러오기~~
      pass

  def forward(self, user, item):
    if self.model_type != 'MLP':
      user_GMF = self.user_embedding_gmf(user)
      item_GMF = self.item_embedding_gmf(item)
      output_GMF = torch.mul(user_GMF, item_GMF) # inner product

    if self.model_type != 'GMF':
      user_MLP = self.user_embedding_mlp(user)
      item_MLP = self.item_embedding_mlp(item)
      #
      concated_input = torch.cat((user_MLP, item_MLP), -1)
      print("concat", concated_input.shape)
      output_MLP = self.mlp_layers(concated_input)

    if self.model_type == 'MLP':
      last_layer = output_MLP
    elif self.model_type == 'GMF':
      last_layer = output_GMF
    else:
      last_layer = torch.cat((output_GMF, output_MLP))

    output = self.last_layer(last_layer)

    print("output", output.shape)
    return output.view(-1)

  def fit(self):
    for epoch in range(self.epochs):




미완성,,,
### 의문점
 ratio가 대략 1: 7의 비율이라고 이해했는데 왜 저자가 사용한 코드는 1: 99인지??

 그리고 train 시에만 negative가 필요한 건데 왜 test.negative가 있는 거지??

나중에 다시 봐야할 것 같은데 너무 오래 끌었더니 물리는 느낌이 든다,,
우선 RecSys24부터 보고 다시 완성하자,,,